In [13]:
%pip install pyspark==3.1.1 pyarrow

Note: you may need to restart the kernel to use updated packages.


In [14]:
import sys
sys.executable

'/usr/local/anaconda3/envs/ada/bin/python'

In [15]:
# Make sure that spark uses the same python distribution to avoid serialization issues due to missing packages
%env SPARK_PYTHON=/usr/local/anaconda3/envs/ada/bin/python
%env SPARK_DRIVER=/usr/local/anaconda3/envs/ada/bin/python

env: SPARK_PYTHON=/usr/local/anaconda3/envs/ada/bin/python
env: SPARK_DRIVER=/usr/local/anaconda3/envs/ada/bin/python


In [48]:
# PySpark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

# Helpers
import os
import json

In [49]:
conf = pyspark.SparkConf().setMaster("local[*]").setAll([
    ('spark.driver.memory','8G'),
    ('spark.driver.maxResultSize', '8G'),
    ('spark.sql.execution.arrow.pyspark.enabled', True),
    ('spark.sql.execution.arrow.maxRecordsPerBatch', 10000),
    ('spark.local.dir', '/tmp')
])

spark = SparkSession.builder.config(conf=conf).config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')
spark

In [18]:
DATA_DIR = 'data/'
QUOTEBANK_DATA_DIR = DATA_DIR + 'quotebank_data/'

In [50]:
JSON_SCHEMA = StructType.fromJson(json.loads(
    '''{  "type": "struct", 
        "fields": [
            {"name": "date", "type": "string", "nullable": true, "metadata": {}}, 
            {"name": "numOccurrences", "type": "long", "nullable": true,"metadata": {}}, 
            {"name": "phase", "type": "string", "nullable": true, "metadata": {}}, 
            {"name": "probas", "type": 
            {"type": "array", "elementType": {"type": "array", "elementType": "string", "containsNull": true}, "containsNull": true}, "nullable": true, "metadata": {}}, 
            {"name": "qids", "type": {"type": "array", "elementType": "string", "containsNull": true}, "nullable": true, "metadata": {}}, 
            {"name": "quotation", "type": "string", "nullable": true, "metadata": {}}, 
            {"name": "quoteID", "type": "string", "nullable": true, "metadata": {}}, 
            {"name": "speaker", "type": "string", "nullable": true, "metadata": {}}, 
            {"name": "urls", "type": {"type": "array", "elementType": "string", "containsNull": true}, "nullable": true, "metadata": {}}
            ]
    }'''))
            

In [51]:
def convert_quotebank_to_parquet():
    for fn in os.listdir(QUOTEBANK_DATA_DIR + 'json'):
        df = spark.read.json(QUOTEBANK_DATA_DIR + 'json/' + fn, schema=JSON_SCHEMA)
        df.write.parquet(QUOTEBANK_DATA_DIR + 'parquet/' + fn[:-4] + 'parquet', 'overwrite')

In [52]:
%%time
convert_quotebank_to_parquet()

CPU times: user 331 ms, sys: 160 ms, total: 491 ms
Wall time: 8min 40s
